In [11]:
import pandas as pd
import random

# Load candidate locations with population
df_population = pd.read_csv(r"C:\Users\user\OneDrive\Desktop\ibtikar_backend-main\Genetic_Algo\clean\final_population_and_credentials_dataset_converted.csv")

# Prepare a list of available coordinates
candidate_locations = list(zip(df_population['latitude'], df_population['longitude']))

# Parameters
N = 5  # Number of relay points per individual (you can set this as needed)
POPULATION_SIZE = 100  # Number of individuals in the GA population

def generate_individual(N, candidate_locations):
    """Generates one individual with N unique locations."""
    return random.sample(candidate_locations, N)

def generate_initial_population(pop_size, N, candidate_locations):
    """Generates the initial GA population."""
    population = []
    for _ in range(pop_size):
        individual = generate_individual(N, candidate_locations)
        population.append(individual)
    return population

# Generate the initial population
initial_population = generate_initial_population(POPULATION_SIZE, N, candidate_locations)

# Optional: display a sample individual
print("Sample Individual:", initial_population)


Sample Individual: [[(36.60611111111111, 3.0877777777777777), (36.73583333333333, 3.3400000000000003), (36.778888888888886, 2.975), (36.714444444444446, 3.106388888888889), (36.68222222222222, 2.877222222222222)], [(36.78444444444444, 3.041111111111111), (36.75972222222222, 3.0580555555555557), (36.79805555555556, 3.0241666666666664), (36.73305555555556, 3.2086111111111117), (36.714444444444446, 3.106388888888889)], [(36.778888888888886, 2.975), (36.60611111111111, 3.0877777777777777), (36.79055555555556, 3.2494444444444444), (36.806666666666665, 3.04), (36.76, 3.0119444444444445)], [(36.79805555555556, 3.0241666666666664), (36.79055555555556, 3.2494444444444444), (36.75083333333333, 2.9833333333333334), (36.676944444444445, 2.9819444444444447), (36.7675, 3.0258333333333334)], [(36.69555555555555, 2.9725), (36.74861111111111, 3.192222222222222), (36.7675, 3.0258333333333334), (36.73361111111111, 3.111111111111111), (36.70111111111111, 3.1775)], [(36.81027777777778, 3.2544444444444443),

In [12]:
import numpy as np
from geopy.distance import geodesic

df_transport = pd.read_csv(r"C:\Users\user\OneDrive\Desktop\ibtikar_backend-main\Genetic_Algo\clean\final_transport_points.csv")
df_study = pd.read_csv(r"C:\Users\user\OneDrive\Desktop\ibtikar_backend-main\Genetic_Algo\clean\cleaned_study_points.csv")

# Convert to lists of coordinates
population_coords = list(zip(df_population['latitude'], df_population['longitude']))
population_values = df_population['population'].values
transport_coords = list(zip(df_transport['latitude'], df_transport['longitude']))
study_coords = list(zip(df_study['latitude'], df_study['longitude']))



In [14]:
study_coords

[(36.7294782, 3.1743236),
 (36.7322506, 3.1840653),
 (36.7243461, 3.184784),
 (36.7208103, 3.1795751),
 (36.7244806, 3.1769703),
 (36.7325544, 3.1732935),
 (36.7321994, 3.1670294),
 (36.7312694, 3.1613689),
 (36.7258806, 3.1508033),
 (36.7278111, 3.1434353),
 (36.7302783, 3.1301597),
 (36.7370321, 3.1136772),
 (36.7407195, 3.1040904),
 (36.7432626, 3.0974131),
 (36.7459931, 3.086813),
 (36.7481604, 3.1970759),
 (36.7470254, 3.1910023),
 (36.7453267, 3.1876677),
 (36.7409744, 3.1860904),
 (36.7373281, 3.1849088),
 (36.742878, 3.0834886),
 (36.7215273, 3.1828188),
 (36.7493038, 3.206485),
 (36.7328166, 3.1246314),
 (36.7357136, 3.1181771),
 (36.7294979, 3.1360516),
 (36.7452182, 3.0922946),
 (36.7730406, 3.2451557),
 (36.7503142, 3.2119921),
 (36.7326363, 3.1245275),
 (36.7535674, 3.2200951),
 (36.7660323, 3.2384262),
 (36.7774533, 3.2501635),
 (36.7589908, 3.2294045),
 (36.7460096, 3.0867914),
 (36.7590229, 3.2293823),
 (36.7774912, 3.2501708),
 (36.7503702, 3.2119574),
 (36.7730613, 3.

In [15]:
def average_pairwise_distance(points):
    """Returns the average distance between all pairs of relay points."""
    dists = [
        geodesic(points[i], points[j]).km
        for i in range(len(points))
        for j in range(i+1, len(points))
    ]
    return np.mean(dists)

def population_score(points, pop_coords, pop_values, radius_km=2.0):
    """Sum of populations within `radius_km` of any relay point."""
    score = 0
    for relay in points:
        for (coord, pop) in zip(pop_coords, pop_values):
            if geodesic(relay, coord).km <= radius_km:
                # “For each relay point, how much population lives within 2 km of it?”
                score += pop
    return score

def inverse_avg_distance_to_nearest(relay_points, target_coords):
    """Average inverse distance from each relay point to nearest target."""
    total = 0
    for relay in relay_points:
        if not target_coords:
            continue
        dists = [geodesic(relay, t).km for t in target_coords]
        total += 1 / (min(dists) + 0.01)  # avoid div by zero
    return total / len(relay_points)


# This function rewards relay points that are close to something important — like transport stops or universities.


def compute_fitness(individual, pop_coords, pop_values, transport_coords, study_coords,
                    max_pop=None, max_spread=None):
    """Compute normalized fitness score."""
    # Raw scores
    spread = average_pairwise_distance(individual)
    pop = population_score(individual, pop_coords, pop_values)
    transport = inverse_avg_distance_to_nearest(individual, transport_coords)
    study = inverse_avg_distance_to_nearest(individual, study_coords)

    # Normalization (divide by precomputed max for fair comparison)
    spread_norm = spread / max_spread if max_spread else 0
    pop_norm = pop / max_pop if max_pop else 0
    transport_norm = transport / 5  # assume avg 5 is good
    study_norm = study / 5  # same

    # Weights
    w1, w2, w3, w4 = 0.4, 0.3, 0.2, 0.1

    fitness = (w1 * spread_norm +
               w2 * pop_norm +
               w3 * transport_norm +
               w4 * study_norm)
    
    return fitness


In [17]:
# Optional: estimate max values by checking many random individuals
def estimate_max_values(candidate_locations, pop_coords, pop_values):
    max_spread = 0
    max_pop = 0
    for _ in range(100):
        individual = random.sample(candidate_locations, 5)
        spread = average_pairwise_distance(individual)
        pop = population_score(individual, pop_coords, pop_values)
        max_spread = max(max_spread, spread)
        max_pop = max(max_pop, pop)
    return max_spread, max_pop

max_spread, max_pop = estimate_max_values(population_coords, population_coords, population_values)
print("max_spread", max_spread)
print("max_pop", max_pop)


max_spread 22.83967172508108
max_pop 811712


In [23]:
def analyze_individual(individual, 
                       pop_coords, pop_values, pop_names,
                       transport_coords, transport_names,
                       study_coords, study_names,
                       max_spread, max_pop,
                       radius_km=2.0):
    
    from geopy.distance import geodesic
    print("="*60)
    print("🧬 Individual (Relay Points):")
    for i, point in enumerate(individual):
        print(f"  Relay {i+1}: ({point[0]:.5f}, {point[1]:.5f})")

    # ---- Spread ----
    spread = average_pairwise_distance(individual)
    spread_norm = spread / max_spread
    print(f"\n📏 Avg Distance Between Points: {spread:.2f} km (Normalized: {spread_norm:.3f})")

    # ---- Population ----
    total_pop = 0
    covered_areas = []
    for relay in individual:
        for (coord, pop, name) in zip(pop_coords, pop_values, pop_names):
            dist = geodesic(relay, coord).km
            if dist <= radius_km:
                total_pop += pop
                covered_areas.append((name, pop, dist))
    pop_norm = total_pop / max_pop
    print(f"\n👥 Population Covered (within {radius_km} km): {total_pop} (Normalized: {pop_norm:.3f})")
    if covered_areas:
        print("Covered Communes:")
        for name, pop, dist in sorted(covered_areas, key=lambda x: -x[1])[:10]:
            print(f"  🏙️ {name} - {pop} people ({dist:.2f} km)")
    else:
        print("  ⚠️ No communes covered within radius.")

    # ---- Transport Proximity ----
    transport_scores = []
    print("\n🚏 Nearest Transport Stops:")
    for i, relay in enumerate(individual):
        dists = [geodesic(relay, t).km for t in transport_coords]
        nearest_idx = np.argmin(dists)
        nearest_dist = dists[nearest_idx]
        transport_scores.append(1 / (nearest_dist + 0.01))
        print(f"  Relay {i+1}: {transport_names[nearest_idx]} ({nearest_dist:.2f} km)")

    transport_score = np.mean(transport_scores)
    transport_norm = transport_score / 5
    print(f"Avg Inverse Distance to Transport: {transport_score:.3f} (Normalized: {transport_norm:.3f})")

    # ---- Study Proximity ----
    study_scores = []
    print("\n🎓 Nearest Study Areas:")
    for i, relay in enumerate(individual):
        dists = [geodesic(relay, s).km for s in study_coords]
        nearest_idx = np.argmin(dists)
        nearest_dist = dists[nearest_idx]
        study_scores.append(1 / (nearest_dist + 0.01))
        print(f"  Relay {i+1}: {study_names[nearest_idx]} ({nearest_dist:.2f} km)")

    study_score = np.mean(study_scores)
    study_norm = study_score / 5
    print(f"Avg Inverse Distance to Study: {study_score:.3f} (Normalized: {study_norm:.3f})")

    # ---- Final Fitness ----
    fitness = compute_fitness(
        individual,
        pop_coords=pop_coords,
        pop_values=pop_values,
        transport_coords=transport_coords,
        study_coords=study_coords,
        max_pop=max_pop,
        max_spread=max_spread
    )

    print(f"\n✅ Final Fitness Score (using compute_fitness): {fitness:.4f}")
    print("="*60 + "\n")


In [21]:
# Assuming you already loaded your CSVs:
# df_population, df_transport, df_study

pop_names = df_population['commune_name'].tolist()
transport_names = df_transport['name'].tolist()
study_names = df_study['commune_name'].tolist()


In [24]:
for i in range(3):
    print(f"\n🔎 Individual {i+1}")
    analyze_individual(
        initial_population[i],
        population_coords,
        population_values,
        pop_names,
        transport_coords,
        transport_names,
        study_coords,
        study_names,
        max_spread,
        max_pop
    )


🔎 Individual 1
🧬 Individual (Relay Points):
  Relay 1: (36.60611, 3.08778)
  Relay 2: (36.73583, 3.34000)
  Relay 3: (36.77889, 2.97500)
  Relay 4: (36.71444, 3.10639)
  Relay 5: (36.68222, 2.87722)

📏 Avg Distance Between Points: 22.53 km (Normalized: 0.986)

👥 Population Covered (within 2.0 km): 428925 (Normalized: 0.528)
Covered Communes:
  🏙️ Bachdjerrah - 93289 people (1.39 km)
  🏙️ Reghaia - 85452 people (0.00 km)
  🏙️ Cheraga - 80824 people (1.90 km)
  🏙️ Bourouba - 71661 people (0.00 km)
  🏙️ Sidi Moussa - 40750 people (0.00 km)
  🏙️ Beni Messous - 36191 people (0.00 km)
  🏙️ Mahelma - 20758 people (0.00 km)

🚏 Nearest Transport Stops:
  Relay 1: Pont El Harrach (14.29 km)
  Relay 2: Dergana Centre (8.17 km)
  Relay 3: Ruisseau (10.47 km)
  Relay 4: Caroubier (2.58 km)
  Relay 5: Ruisseau (19.62 km)
Avg Inverse Distance to Transport: 0.145 (Normalized: 0.029)

🎓 Nearest Study Areas:
  Relay 1: CFPA Houari Boumediene (0.03 km)
  Relay 2: Centre de formation professionnel (0.54 

### let's start the algo 

In [25]:
# Assuming:
# N = number of relay points you want (e.g. 5)
# pop_size = number of individuals in the population (e.g. 100)
# candidate_locations = list of all possible (lat, lon) from communes

N = 5
pop_size = 100
candidate_locations = population_coords  # from df_population

population = generate_initial_population(pop_size, N, candidate_locations)
population

[[(36.73361111111111, 3.111111111111111),
  (36.75305555555556, 2.8877777777777776),
  (36.778888888888886, 2.975),
  (36.59944444444445, 2.9944444444444445),
  (36.78444444444444, 3.041111111111111)],
 [(36.78444444444444, 3.0591666666666666),
  (36.75138888888889, 3.049722222222222),
  (36.72611111111112, 3.182777777777777),
  (36.69555555555555, 2.9725),
  (36.59944444444445, 2.9944444444444445)],
 [(36.73277777777778, 3.0477777777777777),
  (36.696666666666665, 3.095277777777778),
  (36.73638888888889, 2.9497222222222224),
  (36.70805555555556, 2.9125),
  (36.59944444444445, 2.9944444444444445)],
 [(36.79805555555556, 3.0241666666666664),
  (36.73361111111111, 3.111111111111111),
  (36.79333333333333, 3.2869444444444444),
  (36.79055555555556, 3.2494444444444444),
  (36.60611111111111, 3.0877777777777777)],
 [(36.676944444444445, 2.9819444444444447),
  (36.68305555555556, 2.9650000000000003),
  (36.81027777777778, 3.2544444444444443),
  (36.74861111111111, 3.192222222222222),
  (36

In [26]:
fitnesses = [
    compute_fitness(ind, population_coords, population_values, transport_coords, study_coords, max_pop, max_spread)
    for ind in population
]


In [27]:
fitnesses 

[0.45465793387367237,
 0.4680719411069251,
 0.3904284966918003,
 0.5798947943475192,
 0.577379063101449,
 0.31787975411761743,
 0.4376619860468689,
 0.4611439205372716,
 0.38749398863029705,
 0.44642296906489304,
 0.49561630610542573,
 0.5344615732432212,
 0.5391947505758636,
 0.5134356780711813,
 0.43066106464702253,
 0.5221337009497007,
 0.4062758953021869,
 0.35801337023982743,
 0.5596239090820014,
 0.5180510783162324,
 0.4442299171366361,
 0.4839395923682523,
 0.5271669281295907,
 0.5296779094738071,
 0.5168248271748102,
 0.5216344078188883,
 0.550585579143609,
 0.43113019298257277,
 0.48786718829573217,
 0.46935767444242027,
 0.505729556066031,
 0.5422840977946848,
 0.5548916062739006,
 0.6303981898052721,
 0.3281791861018991,
 0.5994906111159327,
 0.5218208759053291,
 0.3905126866700282,
 0.5672213356975329,
 0.556240151176272,
 0.4441453020250399,
 0.5655386800315663,
 0.5190116599147038,
 0.4324270599241341,
 0.5278466889350467,
 0.4730750954550841,
 0.5495784271057007,
 0.5162

In [28]:
def tournament_selection(population, fitnesses, k=3):
    selected = []
    for _ in range(2):  # select 2 parents
        tournament = random.sample(list(zip(population, fitnesses)), k)
        winner = max(tournament, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected


In [29]:
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2


In [30]:
def mutate(individual, available_points, mutation_rate=0.1):
    new_ind = individual.copy()
    for i in range(len(new_ind)):
        if random.random() < mutation_rate:
            new_ind[i] = random.choice(available_points)
    return new_ind


In [32]:
import random

def evolve_population(population, pop_coords, pop_values, transport_coords, study_coords,
                      max_spread, max_pop, generations=50, mutation_rate=0.1):

    best_fitness = 0
    best_individual = None

    for gen in range(generations):
        fitnesses = [
            compute_fitness(ind, pop_coords, pop_values, transport_coords, study_coords, max_pop, max_spread)
            for ind in population
        ]

        # Keep best
        best_idx = max(range(len(fitnesses)), key=lambda i: fitnesses[i])
        if fitnesses[best_idx] > best_fitness:
            best_fitness = fitnesses[best_idx]
            best_individual = population[best_idx]
            print(f"✅ Generation {gen} - New Best Fitness: {best_fitness:.4f}")

        # Next generation
        new_population = []
        while len(new_population) < len(population):
            parent1, parent2 = tournament_selection(population, fitnesses)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, pop_coords, mutation_rate)
            child2 = mutate(child2, pop_coords, mutation_rate)
            new_population.extend([child1, child2])

        # Trim excess if any
        population = new_population[:len(population)]

    return best_individual, best_fitness


In [33]:
best_ind, best_fit = evolve_population(
    initial_population,
    population_coords,
    population_values,
    transport_coords,
    study_coords,
    max_spread,
    max_pop,
    generations=50,
    mutation_rate=0.15
)



✅ Generation 0 - New Best Fitness: 0.6915
✅ Generation 1 - New Best Fitness: 0.7418
✅ Generation 2 - New Best Fitness: 0.7878
✅ Generation 3 - New Best Fitness: 0.7915
✅ Generation 5 - New Best Fitness: 0.8018
✅ Generation 6 - New Best Fitness: 0.8092
✅ Generation 10 - New Best Fitness: 0.8127


In [34]:
analyze_individual(
    best_ind,
    population_coords,
    population_values,
    pop_names,
    transport_coords,
    transport_names,
    study_coords,
    study_names,
    max_spread,
    max_pop
)

🧬 Individual (Relay Points):
  Relay 1: (36.60611, 3.08778)
  Relay 2: (36.74861, 3.19222)
  Relay 3: (36.60611, 3.08778)
  Relay 4: (36.71167, 2.84222)
  Relay 5: (36.81028, 3.25444)

📏 Avg Distance Between Points: 21.95 km (Normalized: 0.961)

👥 Population Covered (within 2.0 km): 297102 (Normalized: 0.366)
Covered Communes:
  🏙️ Bordj El Kiffan - 151950 people (0.00 km)
  🏙️ Zeralda - 51552 people (0.00 km)
  🏙️ Sidi Moussa - 40750 people (0.00 km)
  🏙️ Sidi Moussa - 40750 people (0.00 km)
  🏙️ El Marsa - 12100 people (0.00 km)

🚏 Nearest Transport Stops:
  Relay 1: Pont El Harrach (14.29 km)
  Relay 2: Bordj El Kiffan - Centre (0.21 km)
  Relay 3: Pont El Harrach (14.29 km)
  Relay 4: Ruisseau (21.82 km)
  Relay 5: Faculté Biomédicale (3.63 km)
Avg Inverse Distance to Transport: 1.019 (Normalized: 0.204)

🎓 Nearest Study Areas:
  Relay 1: CFPA Houari Boumediene (0.03 km)
  Relay 2: Bordj El Kiffan - Centre (0.21 km)
  Relay 3: CFPA Houari Boumediene (0.03 km)
  Relay 4: CFPA Hadjar